# SBI Pretrained
Load prerun SBI simulations into posterior object

In [1]:
import torch
import numpy as np
import dill
import matplotlib.pyplot as plt
import pandas as pd
import sbi.utils as utils
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
from sbi.inference.base import infer
from scipy import interpolate


In [6]:
prerun_data_file = open('../../data/beta/prerun_simulations/01122021_084255/dpl_list_beta_event_01122021_084255.pkl', 'rb')
prior_file = open('../../data/beta/prerun_simulations/01122021_084255/prior_beta_event_01122021_084255.pkl', 'rb')
theta_samples_file = open('../../data/beta/prerun_simulations/01122021_084255/theta_samples_beta_event_01122021_084255.pkl', 'rb')
prior_dict_file = open('../../data/beta/prerun_simulations/01122021_084255/prior_dict_beta_event_01122021_084255.pkl', 'rb')

prerun_data = dill.load(prerun_data_file)
prerun_data = torch.stack(prerun_data).float()

prior = dill.load(prior_file)
theta_samples = dill.load(theta_samples_file)
prior_dict = dill.load(prior_dict_file)
param_names = list(prior_dict.keys())

prerun_data_file.close()
prior_file.close()
theta_samples_file.close()
prior_dict_file.close()

#Load example alpha events
conv = np.loadtxt('../../data/beta/alpha_event_grant/rACC_flipCONV_r31_avg.txt', delimiter=',')
no_conv = np.loadtxt('../../data/beta/alpha_event_grant/rACC_flipNO_CONV_r31_avg.txt', delimiter=',')

window = np.arange(250,750)
scaling = 1e11
conv = conv[window] * scaling
no_conv = no_conv[window] * scaling

num_pts = 10001

old_x = np.linspace(0,1,len(conv))
new_x = np.linspace(0,1,num_pts)

conv_interp_func = interpolate.interp1d(old_x, conv, kind='cubic')
conv_interp = conv_interp_func(new_x)

no_conv_interp_func = interpolate.interp1d(old_x, no_conv, kind='cubic')
no_conv_interp = no_conv_interp_func(new_x)


In [3]:
inference = SNPE(prior=prior)
inference = inference.append_simulations(theta_samples, prerun_data)

In [4]:
density_estimator = inference.train()
posterior = inference.build_posterior(density_estimator)

/users/ntolley/.local/lib/python3.7/site-packages/torch/autograd/__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


Neural network successfully converged after 161 epochs.


In [8]:
samples = posterior.sample((1000,), x=conv_interp)
#log_probability = posterior.log_prob(samples, x=observation)
_ = utils.pairplot(samples, fig_size=(20,20), labels=param_names)


Error in callback <function flush_figures at 0x7f307c5c78c0> (for post_execute):


KeyboardInterrupt: 